In [1]:
import tensorflow as tf
import keras
from keras import backend as K
tf.__version__

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [2]:
import pathlib
import itertools
import os
import numpy as np
import pandas as pd

In [3]:
from custom_models import create_unet, create_autoencoder

In [5]:
dataset_folder = r"" # Data containing spacenet 
out_folder = r"" # Set to desired output folder location

BATCH_SIZE = 64
EPOCHS = 30
IMAGE_SIZE = (256,256)
RANDOM_SEED = 101
KERNELS = [3,5,7]
DILATIONS = [1,2,3]
WEIGHTS = ["GlorotNormal","NovelMethod",]
combinations = list(itertools.product(KERNELS,DILATIONS,WEIGHTS))
AUTOTUNE = tf.data.AUTOTUNE

if not os.path.isdir(out_folder):
    os.mkdir(out_folder)
data_dir = pathlib.Path(dataset_folder)

In [7]:
image_count = len(list(data_dir.glob('images/*.jpg')))
images_ds = tf.data.Dataset.list_files(str(data_dir/'images/*'), shuffle=False)
labels_ds = tf.data.Dataset.list_files(str(data_dir/'labels/*'), shuffle=False)
dataset = tf.data.Dataset.zip((images_ds, labels_ds))
dataset = dataset.shuffle(image_count, seed = RANDOM_SEED, reshuffle_each_iteration=False)
val_size = 5000
test_size = 1000
train_ds = dataset.skip(val_size+test_size)
val_ds = dataset.skip(test_size).take(val_size)
test_ds = dataset.take(test_size)
print(dataset.cardinality(),train_ds.cardinality())

tf.Tensor(23028, shape=(), dtype=int64) tf.Tensor(17028, shape=(), dtype=int64)


In [8]:
def process_data(img_path,lab_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.keras.layers.Rescaling(1./255)(img)
    lab = tf.io.read_file(lab_path)
    lab = tf.io.decode_jpeg(lab, channels=1)
    lab = tf.image.resize(lab, IMAGE_SIZE)
    return img, lab
train_ds_mapped = train_ds.map(process_data,num_parallel_calls=AUTOTUNE)
val_ds_mapped = val_ds.map(process_data,num_parallel_calls=AUTOTUNE)
test_ds_mapped = test_ds.map(process_data,num_parallel_calls=AUTOTUNE)

In [9]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=(image_count+1000),reshuffle_each_iteration=True)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds
train_ds_ready = configure_for_performance(train_ds_mapped)
val_ds_ready = configure_for_performance(val_ds_mapped)

In [10]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# def iou(y_true, y_pred):
#     ''' Adapted from https://github.com/IdanC1s2/Spacenet-Building-Detection/blob/431129537fac0f9b98393bdd520e4b032f05c453/Models/Train_Model3Band.py '''
#     y_true_f = K.cast(K.flatten(y_true), dtype='float32')
#     y_pred_f = K.cast(K.flatten(y_pred), dtype='float32')
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + K.epsilon())

# def iou_loss(y_true, y_pred):
#     return -iou(y_true, y_pred)

def iou_hard(y_true, y_pred):
    y_true_f = K.cast(K.greater(K.flatten(y_true), 0.5), dtype='float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), dtype='float32')
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + K.epsilon())

def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [ ]:
for KERNEL, DILATION, WEIGHT in combinations:
    print(KERNEL, DILATION, WEIGHT)
    model = create_unet(IMAGE_SIZE+(3,), KERNEL, DILATION, WEIGHT)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
                  loss=tf.keras.losses.BinaryCrossentropy(), metrics=[f1,iou_hard])
    model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1', patience=10, min_delta = 0.0001, mode='max', restore_best_weights=True)
    history_1 = model.fit(x=train_ds_ready, validation_data=val_ds_ready, epochs=EPOCHS, callbacks=[model_early_stopping])
    test_loss, test_f1, test_iou_hard = model.evaluate(test_ds_mapped.batch(BATCH_SIZE))
    hist_df = pd.DataFrame(history_1.history)
    hist_df.loc["Test"] = [0,0,0, test_loss, test_f1, test_iou_hard]
    hist_df.to_csv(os.path.join(out_folder,f"history_k{KERNEL}_d{DILATION}_{WEIGHT}.csv"))

    test_preds = model.predict(test_ds_mapped.batch(BATCH_SIZE))
    names = [tf.strings.split(f, os.path.sep)[-1][-1].numpy() for f in test_ds.take(test_size)]
    folder_path = os.path.join(out_folder,f"k{KERNEL}_d{DILATION}_{WEIGHT}")
    if not os.path.isdir(folder_path):
        os.mkdir(folder_path)
    for name, test in zip(names, test_preds):
        file_path = os.path.join(folder_path,f"pred_{os.path.splitext(name.decode())[0]}.npy")
        np.save(file_path, test)